# Inventory Management Environments

> To be written.

In [ ]:
#| default_exp agents.newsvendor.erm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import logging

from abc import ABC, abstractmethod
from typing import Union
import numpy as np


from ddopnew.envs.base import BaseEnvironment
from ddopnew.agents.base import BaseAgent
from ddopnew.utils import MDPInfo, Parameter, DatasetWrapper
from ddopnew.loss_functions import TorchQuantileLoss

from ddopnew.dataloaders.base import BaseDataLoader

import torch

In [ ]:
#| export

class SGDBaseAgent(BaseAgent):

    train_mode = "epochs_fit"
    
    def __init__(self, 
            environment_info: MDPInfo,
            dataloader: BaseDataLoader,
            optimizer_params: dict = {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}, # other optimizers: "SGD", "RMSprop"
            learning_rate_scheduler = None, # TODO add base class for learning rate scheduler for typing
            dataloader_params: dict = {"batch_size": 32, "shuffle": True},
            device: str = "cpu" # "cuda" or "cpu"
            ):

        self.device = device
        
        self.set_dataloader(dataloader, dataloader_params)
        self.set_model()
        self.set_loss_function()
        self.set_optimizer(optimizer_params)
        self.set_learning_rate_scheduler(learning_rate_scheduler)

        super().__init__(environment_info)

    def set_dataloader(self, dataloader, dataloader_params):
        dataset = DatasetWrapper(dataloader)
        self.dataloader = torch.utils.data.DataLoader(dataset, **dataloader_params)

    @abstractmethod
    def set_loss_function(self):
        pass

    @abstractmethod
    def set_model(self):
        pass

    def set_optimizer(self, optimizer_params):
        optimizer = optimizer_params["optimizer"]
        optimizer_params_copy = optimizer_params.copy()
        del optimizer_params_copy["optimizer"]

        if optimizer == "Adam":
            self.optimizer = torch.optim.Adam(self.model.parameters(), **optimizer_params_copy)
        elif optimizer == "SGD":
            self.optimizer = torch.optim.SGD(self.model.parameters(), **optimizer_params_copy)
        elif optimizer == "RMSprop":
            self.optimizer = torch.optim.RMSprop(self.model.parameters(), **optimizer_params_copy)
        else:
            raise ValueError(f"Optimizer {optimizer} not supported")
        
    def set_learning_rate_scheduler(self, learning_rate_scheduler):
        if learning_rate_scheduler is not None:
            raise NotImplementedError("Learning rate scheduler not implemented yet")
        else:
            self.learning_rate_scheduler = None

    def fit_epoch(self, X, Y):

        device = next(self.model.parameters()).device
        self.model.train()
        total_loss=0

        for i, output in enumerate(self.dataloader):
            
            X, y = output

            # convert X and y to float32
            X = X.type(torch.float32)
            y = y.type(torch.float32)
            
            X, y = X.to(device), y.to(device)

            self.optimizer.zero_grad()

            y_pred = self.model(X)

            if self.loss_function_params==None:
                loss = self.loss_function(y_pred, y)
            else:
                loss = self.loss_function(y_pred, y, **self.loss_function_params) # TODO: add reduction param when defining loss function

            loss.backward()
            self.optimizer.step()
        
            total_loss += loss.item()
        
        self.model.eval()
        
        return total_loss

    def draw_action(self, X):
        y_pred = self.predict(X)
        return y_pred

    def predict(self, X):

        # TODO handle if X is larger than some size, then split into batches

        device = next(self.model.parameters()).device
        self.model.eval()

        with torch.no_grad():
            X = torch.tensor(X, dtype=torch.float32).to(device)
            y_pred = self.model(X)


        y_pred = y_pred.cpu().numpy()

        return y_pred

    def train(self):
        self.model.train()

    def eval(self):
        self.model.eval()

    def to(self, device):
        self.model.to(device)

In [ ]:
#| export

class NVBaseAgent(SGDBaseAgent):

    def __init__(self, 
        environment_info: MDPInfo,
        dataloader: BaseDataLoader,
        cu: Union[np.ndarray, Parameter],
        co: Union[np.ndarray, Parameter],
        optimizer_params: dict = {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}, # other optimizers: "SGD", "RMSprop"
        learning_rate_scheduler = None, # TODO add base class for learning rate scheduler for typing
        dataloader_params: dict = {"batch_size": 32, "shuffle": True},
        device: str = "cpu" # "cuda" or "cpu"
        ):

        self.sl = cu / (cu + co) # ensure this works if cu and co are Parameters

        super().__init__(environment_info, dataloader, optimizer_params, learning_rate_scheduler, dataloader_params, device)


    def set_loss_function(self):

        self.loss_function_params = {"quantile": self.sl}
        self.loss_function = TorchQuantileLoss(reduction="mean")
        
        logging.debug(f"Loss function set to {self.loss_function}")
        

In [ ]:
#| export

class NewsvendorlERMAgent(NVBaseAgent):

    def __init__(self, 
        environment_info: MDPInfo,
        dataloader: BaseDataLoader,
        cu: Union[np.ndarray, Parameter],
        co: Union[np.ndarray, Parameter],
        optimizer_params: dict = {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}, # other optimizers: "SGD", "RMSprop"
        learning_rate_scheduler = None, # TODO add base class for learning rate scheduler for typing
        model_params: dict = {"input_size": 1, "output_size": 1, "relu_output": False}, #
        dataloader_params: dict = {"batch_size": 32, "shuffle": True},
        device: str = "cpu" # "cuda" or "cpu"
        ):

        self.model_params = model_params

        super().__init__(environment_info, dataloader, cu, co, optimizer_params, learning_rate_scheduler, dataloader_params, device)
    
    def set_model(self):
        from ddopnew.approximators import LinearModel
        self.model = LinearModel(**self.model_params)

In [ ]:
from ddopnew.envs.inventory import NewsvendorEnv
from ddopnew.dataloaders.tabular import XYDataLoader
from ddopnew.experiment_functions import run_experiment, test_agent

In [ ]:
val_index_start = 800 #90_000
test_index_start = 900 #100_000

X = np.random.rand(1000, 2)
Y = np.random.rand(1000, 1)

dataloader = XYDataLoader(X, Y, val_index_start, test_index_start)

environment = NewsvendorEnv(
    dataloader = dataloader,
    underage_cost = 0.42857,
    overage_cost = 1.0,
    gamma = 0.999,
    horizon_train = 365,
)

agent = NewsvendorlERMAgent(environment.mdp_info,
                            dataloader,
                            cu=np.array([0.42857]),
                            co=np.array([1.0]),
                            optimizer_params= {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}, # other optimizers: "SGD", "RMSprop"
                            learning_rate_scheduler = None, # TODO add base class for learning rate scheduler for typing
                            model_params = {"input_size": 2, "output_size": 1, "relu_output": False}, #
                            dataloader_params={"batch_size": 32, "shuffle": True},
                            device = "cpu" # "cuda" or "cpu"
)

environment.test()
agent.eval()


R, J = test_agent(agent, environment)

print(R, J)

run_experiment(agent, environment, 2, run_id = "test") # fit agent via run_experiment function

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

Step 100
-18.841829378284025 -17.90628799781712
Step 100
Step 100
Step 100
Step 100
-15.13178600787327 -14.426744918943673


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()